#(NOT FINISHED/FUNCTIONAL)

Text Classification with Attentional RNN!

HIGHLY BASED ON THE LINKS BELOW:

https://richliao.github.io/supervised/classification/2016/12/26/textclassifier-RNN/

https://machinelearningmastery.com/predict-sentiment-movie-reviews-using-deep-learning/

In [4]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.engine.topology import Layer
from keras import initializers

import tensorflow as tf

Using TensorFlow backend.


In [0]:
class AttLayer(Layer):
  def __init__(self, **kwargs):
    print (kwargs)
    self.init = initializers.get('normal')
    super(AttLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
      print input_shape
      assert len(input_shape)==3
      print input_shape
      self.W = self.init((input_shape[-1],))
      self.trainable_weights = [self.W]
      super(AttLayer, self).build(input_shape)
    
    def call(self, x, mask=None):
      eij = tf.layers.tanh(tf.matmul(x, self.W))
      
      ai = tf.exp(eij)
      weights = ai/tf.sum(ai, axis=1).dimshufflw(0, 'x')
      
      weighted_input = x*weights.dimshuffle(0, 1, 'x')
      return weighted_input.sum(axis=1)
    
    def get_output_shape_for(self, input_shape):
      return (input_shape[0], input_shape[-1])
      

In [0]:
# SELECT VOCABULARY MAXIMUN SIZE
top_words = 20000
(X_train, y_train), (X_val, y_val) = imdb.load_data(num_words=top_words)

In [0]:
# PADDING DATA ACCORDING TO MAXIMUN SENTENCE LENGTH
max_words = 1000
X_t = sequence.pad_sequences(X_train, maxlen=max_words)
X_v = sequence.pad_sequences(X_val, maxlen=max_words)

# CONVERTING TO ONE HOT
depth = 2
y_t = to_categorical(y_train.astype(int), depth)
y_v = to_categorical(y_val.astype(int), depth)

In [8]:
sequence_input = Input(shape=(max_words,))
embedded_sequences = Embedding(top_words, 100, input_length=max_words)(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
l_att = AttLayer()(l_lstm)
preds = Dense(2, activation="softmax")(l_att)
model = Model(sequence_input, preds)


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

{}


In [9]:
print("Model fitting - Bidirectional LSTM")
model.summary()

Model fitting - Bidirectional LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 2,161,202
Trainable params: 2,161,202
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_t, y_t, epochs=1, steps_per_epoch=100, verbose=1)

Epoch 1/1
